<a href="https://colab.research.google.com/github/gojiteji/Word2Vec_trial/blob/master/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import re
import time
import torch.optim as optimizers
from sklearn.utils import shuffle
import urllib.request
import zipfile
import re
!apt-get install -y mecab libmecab-dev mecab-ipadic
!apt-get install -y mecab-ipadic-utf8
!apt-get install -y libc6-dev build-essential
!pip3 install mecab-python3

In [0]:
# zipファイルダウンロード
url = 'https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip'
zip = '794_ruby_4237.zip'
urllib.request.urlretrieve(url, zip)

with zipfile.ZipFile(zip, 'r') as myzip:
    myzip.extractall()
    for myfile in myzip.infolist():
        filename = myfile.filename
        with open(filename, encoding='sjis') as file:
            text = file.read()

#いらない文字列の処理
text = re.split('\-{5,}',text)[2]
text = re.split('底本：',text)[0]
text = text.replace('|', '')
text = re.sub('《.+?》', '', text)
text = re.sub('［＃.+?］', '',text)
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)
text=re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
text=re.sub('RT', "", text)
text=re.sub('\u3000', "", text)
text=re.sub('」', "", text)
text=re.sub('「', "", text)
text=re.sub('、', "", text)
text=re.sub('。', "", text)
text=re.sub('…', "", text)
text=re.sub('一', "", text)
text=re.sub(' ', "", text)
text=re.sub('　', "", text)
text=re.sub(r'[︰-＠]', "", text)#全角記号
#text=re.sub('\n', " ", text)#改行文字

In [0]:
#形態素に分解
import MeCab
mecab = MeCab.Tagger('')
malist = mecab.parse(text)

#必要なだけ抽出
words=[]
splited=malist.split()
for i in range(0,len(splited),2):
    if(splited[i]=='EOS'):
        break
    if(re.split(',',splited[i+1])[0]=="副詞"):
        words.append(splited[i])
    elif(re.split(',',splited[i+1])[0]=="動詞"):
        words.append(splited[i])
    elif(re.split(',',splited[i+1])[0]=="名詞"):
        words.append(splited[i])
    elif(re.split(',',splited[i+1])[0]=="形容詞"):
        words.append(splited[i])

In [0]:
w2i={}
i2w={}
_words=set()#値が被らない構造
_words.update(words)

w2i = {w: (i)for i, w in enumerate(_words)}
i2w = {i: w for w, i in w2i.items()}

In [0]:
def get_indexes(word):
    global words
    indexes=[i for i, _x in enumerate(words) if _x == word]
    return indexes

In [90]:
!pip install tqdm
from tqdm import tqdm

In [0]:
x=torch.Tensor(list(w2i.values()))#入力データ行列
y=torch.zeros(len(x),len(x))#i行目の単語i2w[i]はj列目の単語i2w[j]とy[i][j]回出てきている。
window_size=5

l=len(x)
for i in tqdm(range(l)):
    for j in range(l):#列
        count = 0 #i番目の単語周辺に単語jが発生する数
        if(j is not i):#自身は除外
            center_indexes = get_indexes(i)
            target_word = i2w[j]
            for center in center_indexes:
                if(center-window_size<0):
                    count=count + words[c0:center+window_size+1].count(word)
                elif(center+window_size+1>l):
                    count=count + words[center-window_size:].count(word)
                else:
                    count=count + words[center-window_size:center+window_size+1].count(word)
        y[i][j] = count
    y[i]=(y[i]-y[i].min())/(y[i].max()-y[i].min())#行ごとに正規化








  0%|          | 1/6818 [00:21<39:50:39, 21.04s/it]

0









  0%|          | 2/6818 [00:41<39:34:51, 20.91s/it]






  0%|          | 3/6818 [01:02<39:22:49, 20.80s/it]






  0%|          | 4/6818 [01:23<39:29:16, 20.86s/it]






  0%|          | 5/6818 [01:43<39:26:51, 20.84s/it]






  0%|          | 6/6818 [02:04<39:19:42, 20.78s/it]






  0%|          | 7/6818 [02:25<39:23:16, 20.82s/it]






  0%|          | 8/6818 [02:46<39:23:46, 20.83s/it]






  0%|          | 9/6818 [03:07<39:25:39, 20.85s/it]






  0%|          | 10/6818 [03:28<39:25:58, 20.85s/it]






  0%|          | 11/6818 [03:48<39:18:29, 20.79s/it]






  0%|          | 12/6818 [04:09<39:16:54, 20.78s/it]






  0%|          | 13/6818 [04:30<39:29:33, 20.89s/it]






  0%|          | 14/6818 [04:51<39:27:49, 20.88s/it]






  0%|          | 15/6818 [05:12<39:29:50, 20.90s/it]






  0%|          | 16/6818 [05:33<39:30:23, 20.91s/it]






  0%|          | 17/6818 [05:54<39:30:16, 20.91s/it]






  0%|          | 18/6818 [06:15<39:26:29, 20.88s/it]






61









  1%|          | 63/6818 [21:59<39:18:48, 20.95s/it]






  1%|          | 64/6818 [22:20<39:30:50, 21.06s/it]






  1%|          | 65/6818 [22:41<39:26:17, 21.02s/it]






  1%|          | 66/6818 [23:02<39:28:34, 21.05s/it]






  1%|          | 67/6818 [23:23<39:25:48, 21.03s/it]






  1%|          | 68/6818 [23:44<39:33:11, 21.10s/it]






  1%|          | 69/6818 [24:05<39:35:10, 21.12s/it]






  1%|          | 70/6818 [24:27<39:37:10, 21.14s/it]






  1%|          | 71/6818 [24:48<39:36:48, 21.14s/it]






  1%|          | 72/6818 [25:09<39:35:16, 21.13s/it]






  1%|          | 73/6818 [25:30<39:39:14, 21.16s/it]






  1%|          | 74/6818 [25:51<39:29:14, 21.08s/it]






  1%|          | 75/6818 [26:12<39:29:56, 21.09s/it]






  1%|          | 76/6818 [26:33<39:24:03, 21.04s/it]






  1%|          | 77/6818 [26:54<39:31:35, 21.11s/it]






  1%|          | 78/6818 [27:16<39:32:43, 21.12s/it]






  1%|          | 79/6818 [27:37<39:28:13, 21.09s/

In [0]:
class EncoderDecoder(nn.Module):
    def __init__(self,input_dim,vec_dim,device='cuda'):
        super().__init__()
        self.device = device
        self.input_dim=input_dim
        self.vec_dim=vec_dim
        self.encoder=Encoder(self.input_dim,self.vec_dim)
        self.decoder=Decoder(self.input_dim,self.vec_dim)
    def forward(self,x):
        if(x in w2i):
            y=w2i(x)#word->id
            y=torch.eye(self.input_dim)[y]#id->onehot
            y=self.encoder(y)#onehot->vec
            y=self.decoder(y)#vec->prob
        return y


class Encoder(nn.Module):
    def __init__(self,input_dim,vec_dim,device='cuda'):
        super().__init__()
        self.device = device
        self.layer=nn.Linear(input_dim,vec_dim)#これがそのままルックアプテーブルになる。

    def forward(self,x):
        y=self.layer
        return y


class Decoder(nn.Module):
    def __init__(self,input_dim,vec_dim,device='cuda'):
        super().__init__()
        self.device = device
        self.layer=nn.Linear(vec_dim,input_dim)
        self.out=nn.Softmax()
    def forward(self,x):
        y=self.layer(x)
        y=self.out(y)
        return y

# 学習

In [0]:
model=EncoderDecoder(len(_words),10)
criterion = nn.CrossEntropyLoss()
optimizer = optimizers.Adam(model.parameters())
def train_step(x, t):
    model.train()
    preds = model(x, t)
    loss = compute_loss()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, preds
for i in range(epochs):
    train_x,train_y=shuffle(x,y)
    for i in range(l)
        train_step(train_x[l],train_y[l])
model.eval()

In [0]:
#word2vecテーブル
print(model.encoder.layer.weight)